# SQL Micro-Brain Fine-tuning on Spider Dataset

This notebook demonstrates how to fine-tune SQL Micro-Brain on the Spider SQL dataset using Google Colab's free T4 GPU.

## Prerequisites
- Google Colab account
- At least 16GB GPU RAM (T4 is sufficient)
- Good internet connection for dataset download

## What you'll learn
- Setting up a Python environment for LLM fine-tuning
- Processing the Spider dataset for instruction tuning
- Fine-tuning a 1.5B parameter model with LoRA
- Running inference on SQL generation tasks

## Time estimates
- Setup: ~10 minutes
- Dataset download and preprocessing: ~15 minutes
- Model training: ~2-4 hours
- Inference demo: ~5 minutes

## 1. Environment Setup

Install required packages and setup the environment. This includes:
- Python packages for transformers, PEFT, datasets
- Git for cloning the repository
- Dependencies for GPU training

In [ ]:
# Install required packages
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install transformers accelerate peft datasets
!pip install huggingface_hub

# Verify GPU availability
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name()}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory // (1024**3)}GB")


## 2. Clone SQL Micro-Brain Repository

Clone the repository and navigate to the project directory.

In [ ]:
# Clone the repository
!git clone https://github.com/neilecm/SQL_MicroBrain.git
%cd SQL_MicroBrain

# List files to verify clone
!ls -la

## 3. Download Spider Dataset

Download the Spider dataset which contains SQL queries paired with natural language questions. The dataset will be automatically extracted to `data/raw/spider/`.

In [ ]:
# Download Spider dataset
!wget -P data/raw/ https://drive.google.com/uc?id=1nOd9T2S8AEq7Qm5WkHh_kwOXrHjdh3iS --no-check-certificate

# Extract the dataset (assuming it's a zip file)
!unzip -q data/raw/spider.zip -d data/raw/

# Verify extraction
!ls -la data/raw/spider/

## 4. Build Training Data

Process the Spider dataset into JSONL format suitable for instruction tuning. This involves:
- Loading Spider JSON files
- Integrating database schemas
- Formatting into chat conversations

In [ ]:
# Run the training data preparation script
!python training/scripts/build_spider_training_data.py

# Check that the training data was created
!ls -la data/processed/train_spider_sqlmb.jsonl
!head -5 data/processed/train_spider_sqlmb.jsonl

## 5. Model Training

Fine-tune the Qwen2.5-Coder-1.5B-Instruct model using LoRA. This process:
- Uses memory-efficient LoRA for parameter reduction
- Optimizes hyperparameters for T4 GPU
- Takes approximately 2-4 hours depending on dataset size

In [ ]:
# Start training (this will take several hours)
!python training/scripts/train_lora_spider.py

# Check that the model was saved
!ls -la models/sql-micro-brain-spider-lora/

## 6. Inference Demo

Test the fine-tuned model on SQL generation tasks. We'll create a simple inference script and demonstrate it with example queries.

In [ ]:
# Create inference demo script
%%writefile inference_demo.py
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from pathlib import Path
import json

def setup_model():
    """Load the fine-tuned model"""
    ROOT = Path(__file__).resolve().parents[0]
    model_path = ROOT / "models" / "sql-micro-brain-spider-lora"
    
    # Load base model
    base_model = "Qwen/Qwen2.5-Coder-1.5B-Instruct"
    model = AutoModelForCausalLM.from_pretrained(
        base_model,
        torch_dtype=torch.float16,
        device_map="auto"
    )
    
    # Load LoRA adapter
    model = PeftModel.from_pretrained(model, str(model_path))
    
    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(str(model_path))
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    
    return model, tokenizer

def generate_sql(model, tokenizer, question, schema_sql, db_name=""):
    """Generate SQL from natural language task and schema"""
    system_prompt = (
        "You are SQL Micro-Brain, an expert PostgreSQL assistant. "
        "Given a natural language task and a database schema, you output a JSON object "
        "with actions, migrations, rls_policies, indexes, queries, error_explanations, "
        "explanations, and safe_to_execute. You MUST produce valid PostgreSQL SQL."
    )
    
    user_content = (
        f"Task: {question}\n\n"
        f"Database: {db_name}\n\n"
        f"Schema:\n{schema_sql}\n"
    )
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_content}
    ]
    
    # Format conversation
    input_text = tokenizer.apply_chat_template(
        messages,
        tokenize=False,
        add_generation_prompt=True
    )
    
    # Tokenize
    inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
    
    # Generate
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=512,
            temperature=0.1,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # Decode response
    generated_text = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    
    return generated_text

if __name__ == "__main__":
    print("Loading model...")
    model, tokenizer = setup_model()
    
    # Example schema (simplified)
    sample_schema = """
    CREATE TABLE student (
        id INTEGER PRIMARY KEY,
        name TEXT,
        major TEXT
    );
    
    CREATE TABLE course (
        id INTEGER PRIMARY KEY,
        name TEXT,
        credits INTEGER
    );
    """
    
    # Test questions
    test_questions = [
        "What are the names of all students majoring in Computer Science?",
        "How many credits does the database course have?",
        "List all courses with more than 3 credits."
    ]
    
    print("\n--- Inference Demo ---\n")
    for question in test_questions:
        print(f"Question: {question}")
        response = generate_sql(model, tokenizer, question, sample_schema, "university")
        print(f"Response: {response}\n")


In [ ]:
# Run the inference demo
!python inference_demo.py

## 7. Export and Save Results

Zip the fine-tuned model and download for later use.

In [ ]:
# Zip the model directory
!zip -r sql-micro-brain-spider-lora.zip models/sql-micro-brain-spider-lora/

# Download the model (in Colab, this will trigger a download)
from google.colab import files
files.download('sql-micro-brain-spider-lora.zip')

## 8. Cleanup and Tips

### Memory Management
- If you run out of memory during training, reduce `per_device_train_batch_size` to 1
- Enable gradient checkpointing (already included) saves memory at cost of speed
- Use mixed precision training (bf16 already enabled)

### Troubleshooting
- If dataset download fails, try alternative download methods
- If training fails with CUDA errors, restart runtime and try again
- Monitor GPU usage with `!nvidia-smi` in a separate cell

### Next Steps
- Evaluate model performance on test set using Spider evaluation script
- Try different LoRA configurations or hyperparameters
- Deploy the model for production use

### Cost Considerations
- Google Colab Pro: ~$10/month for faster GPUs and longer runtimes
- Training time can be reduced with A100 GPUs (Colab Pro+)
- Consider using cloud instances for production training